In [ ]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
import os
sys.path.append('../python/')

#set up matplotlib
os.environ['MPLCONFIGDIR'] = '../mplstyles'
print(os.environ['MPLCONFIGDIR'])
import matplotlib as mpl
from matplotlib import pyplot as plt
#got smarter about the mpl config: see mplstyles/ directory
plt.style.use('standard')
print(mpl.__version__) 
print(mpl.get_configdir())


#fonts
# Set the font dictionaries (for plot title and axis titles)
title_font = {'fontname':'Arial', 'size':'16', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'32'}
legend_font = {'fontname':'Arial', 'size':'22'}

#fonts global settings
mpl.rc('font',family=legend_font['fontname'])


#set up numpy
import numpy as np

# Summary

Fits have been done in notebooks `edelweiss_C.ipynb` and `Qwidth_confirm.ipynb` which give the functional forms of the "C" parameter as a function of energy and the correction to this parameter, called C$^{\prime}$, due to multiple scattering. 

The fits in each case have constant components and a slope that are correlated, to find "bounding curves" of these functions given the statistical errors may be easiest with an MCMC fitting approach. That is discussed here. 

# Multiple Scatters

For the multiple scatters fitting that was first accomplieshed in the notebook `Qwidth_confirm.ipynb`, the goal would be to expand on the central value for each parameter of the fit, taking into account correlation. From the previous fit we have the following parameters from the `lmfit` output. 


Parameter name|Parameter value|Parameter uncertainty
:-|:-|:-
C$_{ms}$|0.0202 | 0.0014 (6.7%)
m | 5.343$\times$10$^{-5}$ | 1.513$\times$10$^{-5}$ (28.3%)
corr(C$_{ms}$,m) | -0.898 | - 


The slope parameter has high fractional uncertainty and the parameters are highly correlated. In this situation fluctuating each of them upward by the 1$\sigma$ level and taking the resulting cuve from those parameters will overestimate the true uncertainty on the measured quadrature correction for multiple scatters, C$^{\prime}$, as a function of energy. The _correlation_ parameter is a dimensionless parameter between -1 and 1 where the magnitude describes the size of the correlation and the sign describes whether it is an _anti_correlation (see [[corrNotes][corrNotes]])

[corrNotes]: https://www.unige.ch/sciences/astro/files/5413/8971/4090/2_Segransan_StatClassUnige.pdf "correlation notes"

We therefore seek a fitting method that will yield the two-dimensional distribution of C$_{ms}$ and m. The method we use is a Markov Chain Monte Carlo (MCMC) technique, as implemented by the `emcee` library [[emcee][emcee]]. 

[emcee]: https://emcee.readthedocs.io/en/v2.2.1/user/line/ "emcee implemntation of MCMC"

We start, below, by getting a version of the high-statistics mulitple-scatter simulation, and the calculation for the single-scatter predicted yield width. 

In [ ]:
import h5py
filename = 'data/sims.h5'
#remove vars
f = h5py.File(filename,'r')

#save the results for the Edw fit
path='{}/'.format('NR')

xE = np.asarray(f[path+'xE'])
qbootsigs_nr_ms = np.asarray(f[path+'qbootsigs'])
qbootsigerrsu_nr_ms = np.asarray(f[path+'qbootsigerrsu'])
qbootsigerrsl_nr_ms = np.asarray(f[path+'qbootsigerrsl'])


f.close()

In [ ]:
#get the NR calculations for our yield band
import fano_calc as fc
Enr,signr = fc.RWCalc(filename='data/res_calc.h5',band='NR',alpha=(1/18.0))

In [ ]:
#let's make a callable out of the predicted single-scatter resolution
import scipy.interpolate as inter

sigQnr_c = inter.InterpolatedUnivariateSpline(Enr, signr, k=3)
sigQnr_c_v = np.vectorize(sigQnr_c)

We take the model we have for the single-scatter yield width, $\sigma_{\mathrm{Q}ss}(E_r)$, and construct a model for the width including all scatters as:

\begin{equation}
\sigma_{\mathrm{Q}} = \sqrt{\sigma_{\mathrm{Q}ss}^2 + C^{\prime2}},
\end{equation}

Where C$^{\prime}$ is modeled as the linear (in recoil energy) function C$_{ms}$ + m$E_r$. 

In [ ]:
#also construct a residual function for use with lmfit
import lmfit as lmf

def residual(params, x, data, eps_data,sigQfunc):
    Cms = params['Cms']
    m = params['m']


    model = np.sqrt(sigQfunc(x)**2 + (Cms+m*x)**2)

    return (data-model) / eps_data

def residual_const(params, x, data, eps_data,sigQfunc):
    Cms = params['Cms']


    model = np.sqrt(sigQfunc(x)**2 + (Cms)**2)

    return (data-model) / eps_data

#do it with lmfit
params = lmf.Parameters()
params.add('Cms', value=0.02)
lmfout = lmf.minimize(residual_const, params, args=(xE, qbootsigs_nr_ms, qbootsigerrsu_nr_ms,sigQnr_c_v))
#print(lmf.fit_report(lmfout))
print('lmfit result--multiples: constant C')
print(lmf.report_fit(lmfout.params))

Cms_const = lmfout.params['Cms'].value

params = lmf.Parameters()
params.add('Cms', value=0.02)
params.add('m', value=0)
lmfout = lmf.minimize(residual, params, args=(xE, qbootsigs_nr_ms, qbootsigerrsu_nr_ms,sigQnr_c_v))
#print(lmf.fit_report(lmfout))
print('lmfit result--multiples: linear C')
print(lmf.report_fit(lmfout.params))

Cms = lmfout.params['Cms'].value
slope = lmfout.params['m'].value

In [ ]:
print(np.shape(xE))
print(xE)
print(np.shape(qbootsigs_nr_ms))
print(qbootsigs_nr_ms)

#get rid of the firs point because it's spurious
#xE = xE[2:]
#qbootsigs_nr_ms = qbootsigs_nr_ms[2:]
#qbootsigerrsu_nr_ms = qbootsigerrsu_nr_ms[2:]

#print(np.shape(xE))
#print(xE)
#print(np.shape(qbootsigs_nr_ms))
#print(qbootsigs_nr_ms)
#print(np.shape(qbootsigerrsu_nr_ms))
#print(qbootsigerrsu_nr_ms)

In [ ]:
# from here on basically following https://emcee.readthedocs.io/en/latest/tutorials/line/

#ACTUALLY
#that above link is for the bleeding edge 3.0 ish version, it is not easy to install and appears to be failing tests
#so use the stable:
#https://emcee.readthedocs.io/en/stable/user/line/#marginalization-uncertainty-estimation
#note several interfaces have changed in the 3.0x version so stick to this example

In [ ]:
def log_likelihood(theta, x, y, yerr,sigfunc):
    Cms, m = theta
    model = np.sqrt(sigfunc(x)**2 + (Cms+m*x)**2)
    #sigma2 = yerr**2 + model**2*np.exp(2*log_f)
    sigma2 = yerr**2
    return -0.5*np.sum((y-model)**2/sigma2 + np.log(sigma2))

In [ ]:
from scipy.optimize import minimize
np.random.seed(42)
nll = lambda *args: -log_likelihood(*args)
initial = np.array([0.0207, 4.73e-5]) + 0.001*np.random.randn(2)
soln = minimize(nll, initial, args=(xE, qbootsigs_nr_ms, qbootsigerrsu_nr_ms,sigQnr_c_v))
Cms_ml, m_ml = soln.x

print("Maximum likelihood estimates:")
print("m = {0:.4f}".format(Cms_ml))
print("b = {0:.3e}".format(m_ml))


plt.errorbar(xE, qbootsigs_nr_ms, yerr=qbootsigerrsu_nr_ms, fmt=".k", capsize=0)
#plt.plot(x0, m_true*x0+b_true, "k", alpha=0.3, lw=3, label="truth")
#plt.plot(x0, np.dot(np.vander(x0, 2), w), "--k", label="LS")
#plt.plot(x0, np.dot(np.vander(x0, 2), [m_ml, b_ml]), ":k", label="ML")
#plt.legend(fontsize=14)
plt.xlim(10, 200)
plt.ylim(0,0.05)
plt.xlabel("E")
plt.ylabel("$\sigma$");

In [ ]:
def log_prior(theta):
    Cms, m = theta
    if 0.00 < Cms < 0.04 and -1e-4 < m < 1e-4:
        return 0.0
    return -np.inf

In [ ]:
def log_probability(theta, x, y, yerr,sigfunc):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr,sigfunc)

In [ ]:
import emcee

ndim, nwalkers = 2, 100
pos = [soln.x + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, \
          args=(xE, qbootsigs_nr_ms, qbootsigerrsu_nr_ms,sigQnr_c_v))

sampler.run_mcmc(pos, 500)

#import emcee
#print(emcee.__version__)
#pos = soln.x + 1e-4*np.random.randn(32, 2)
#nwalkers, ndim = pos.shape

#sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, \
#          args=(xE, qbootsigs_nr_ms, qbootsigerrsu_nr_ms,sigQnr_c_v))
#sampler.run_mcmc(pos, 5000, progress=True)
#chain = sampler.run_mcmc(pos, 5000)

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
#samples = sampler.get_chain()
samples = sampler.chain
labels = ["Cms", "m"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(np.transpose(samples[:, :, i]), "k", alpha=0.3)
    ax.set_xlim(0, np.shape(samples)[1])
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
import corner
print(np.shape(samples))
samples = sampler.chain[:, 100:, :].reshape((-1, ndim))
print(np.shape(samples))
fig = corner.corner(samples, labels=["$Cms$", "$m$"],
                      truths=[Cms, slope])

In [ ]:
import matplotlib.pyplot as pl
#xl = np.array([10, 200])
xl = np.linspace(10, 200, 500)
#print(xl)
for Cms_em, m_em in samples[np.random.randint(len(samples), size=100)]:
    pl.plot(xl, np.sqrt(sigQnr_c_v(xl)**2+(Cms_em+m_em*xl)**2), color="k", alpha=0.1)
pl.plot(xl, np.sqrt(sigQnr_c_v(xl)**2+(Cms+slope*xl)**2), color="r", lw=2, alpha=0.8)
#pl.errorbar(x, y, yerr=yerr, fmt=".k")
pl.xlim(10, 200)
pl.ylim(0,0.05)
pl.errorbar(xE, qbootsigs_nr_ms, yerr=qbootsigerrsu_nr_ms, fmt=".k", capsize=0)

In [ ]:
print(len(samples))

In [ ]:
sampsize=100
evec = np.zeros((np.shape(xl)[0],sampsize))
#print(samples[np.random.randint(len(samples), size=sampsize)])
i=0
for Cms_em, m_em in samples[np.random.randint(len(samples), size=sampsize)]:
    v = np.sqrt(sigQnr_c_v(xl)**2+(Cms_em+m_em*xl)**2)
    #print(np.std(v))
    #print(np.mean(v))
    cent = np.sqrt(sigQnr_c_v(xl)**2+(Cms+slope*xl)**2)
    evec[:,i] = v
    #print(i)
    #print(np.mean(v)+np.std(v))
    #print(np.mean(v)-np.std(v))
    i=i+1
   
upvec = np.mean(evec,axis=1)+np.std(evec,axis=1)
dnvec = np.mean(evec,axis=1)-np.std(evec,axis=1)

In [ ]:
pl.plot(xl, np.sqrt(sigQnr_c_v(xl)**2+(Cms+slope*xl)**2), color="r", lw=2, alpha=0.8)
pl.plot(xl, upvec, color="r",linestyle="--", lw=2, alpha=0.8)
pl.plot(xl, dnvec, color="r",linestyle="--", lw=2, alpha=0.8)
#pl.errorbar(x, y, yerr=yerr, fmt=".k")
pl.xlim(10, 200)
pl.ylim(0,0.05)
pl.errorbar(xE, qbootsigs_nr_ms, yerr=qbootsigerrsu_nr_ms, fmt=".k", capsize=0)

In [ ]:
import h5py

#save the results for the MS fit
path='{}/{}/'.format('mcmc','multiples')

filename = 'data/mcmc_fits.h5'

#remove vars
f = h5py.File(filename,'a')
exCms = path+'Cms' in f
exm = path+'m' in f
exsamp = path+'samples' in f
exsampsize = path+'sampsize' in f
exEr = path+'Er' in f
exCsig_u = path+'Csig_u' in f
exCsig_l = path+'Csig_l' in f
exSigss = path+'Sigss' in f

if exCms:
  del f[path+'Cms']
if exm:
  del f[path+'m']
if exsamp:
  del f[path+'samples']
if exsampsize:
  del f[path+'sampsize']
if exEr:
  del f[path+'Er']
if exCsig_u:
  del f[path+'Csig_u']
if exCsig_l:
  del f[path+'Csig_l']
if exSigss:
  del f[path+'Sigss']

dset = f.create_dataset(path+'Cms',np.shape(Cms),dtype=np.dtype('float64').type)
dset[...] = Cms
dset = f.create_dataset(path+'m',np.shape(slope),dtype=np.dtype('float64').type)
dset[...] = slope
dset = f.create_dataset(path+'samples',np.shape(samples),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = samples
dset = f.create_dataset(path+'sampsize',np.shape(sampsize),dtype=np.dtype('float64').type)
dset[...] = sampsize
dset = f.create_dataset(path+'Er',np.shape(xl),dtype=np.dtype('float64').type, \
compression="gzip")
dset[...] = xl
dset = f.create_dataset(path+'Csig_u',np.shape(upvec),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = upvec
dset = f.create_dataset(path+'Csig_l',np.shape(dnvec),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = dnvec
dset = f.create_dataset(path+'Sigss',np.shape(xl),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = sigQnr_c_v(xl)

f.close()

# Experimental Yield Widening (EDELWEISS Fit)

This procedure can also be done for the fit (originally in the notebook `edelweiss_C.ipynb`) that extracts the uncertainty and central value for the experimentally observed widening parameter, C. 

In [ ]:
# import data from Edelweiss
import pandas as pds
res_data = pds.read_csv("data/edelweiss_NRwidth_GGA3_data.txt", skiprows=1, \
                       names=['E_recoil', 'sig_NR', 'E_recoil_err', 'sig_NR_err'], \
                       delim_whitespace=True)

resER_data = pds.read_csv("data/edelweiss_ERwidth_GGA3_data.txt", skiprows=1, \
                         names=['E_recoil', 'sig_ER', 'sig_ER_err'], \
                         delim_whitespace=True)

resER_data = resER_data.sort_values(by='E_recoil')

print (res_data.head(10))
E_recoil = res_data["E_recoil"]
sig_NR = res_data["sig_NR"]
sig_NR_err = res_data['sig_NR_err']

In [ ]:
#some of the fit points are known to be "bad" because of the presence of inelastic scattering
#to fix that ignore the first two points

#do it with lmfit
params = lmf.Parameters()
params.add('Cms', value=0.04)
lmfout = lmf.minimize(residual_const, params, args=(E_recoil[2::], sig_NR[2::], sig_NR_err[2::],sigQnr_c_v))
#print(lmf.fit_report(lmfout))
print('lmfit result--multiples: constant C')
print(lmf.report_fit(lmfout.params))

Cms_const = lmfout.params['Cms'].value

params = lmf.Parameters()
params.add('Cms', value=0.04)
params.add('m', value=0)
lmfout = lmf.minimize(residual, params, args=(E_recoil[2::], sig_NR[2::], sig_NR_err[2::],sigQnr_c_v))
#print(lmf.fit_report(lmfout))
print('lmfit result--multiples: linear C')
print(lmf.report_fit(lmfout.params))

Cms = lmfout.params['Cms'].value
slope = lmfout.params['m'].value

In [ ]:
def new_log_prior(theta):
    Cms, m = theta
    if 0.02 < Cms < 0.05 and -1e-3 < m < 1e-3:
        return 0.0
    return -np.inf

In [ ]:
def new_log_probability(theta, x, y, yerr,sigfunc):
    lp = new_log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr,sigfunc)

In [ ]:
ndim, nwalkers = 2, 100
pos = [[Cms,slope] + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, new_log_probability, \
          args=(E_recoil[2::], sig_NR[2::], sig_NR_err[2::],sigQnr_c_v))

sampler.run_mcmc(pos, 500)

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
#samples = sampler.get_chain()
samples = sampler.chain
labels = ["Cms", "m"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(np.transpose(samples[:, :, i]), "k", alpha=0.3)
    ax.set_xlim(0, np.shape(samples)[1])
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
print(np.shape(samples))
samples = sampler.chain[:, 100:, :].reshape((-1, ndim))
print(np.shape(samples))
fig = corner.corner(samples, labels=["$Cms$", "$m$"],
                      truths=[Cms, slope])

In [ ]:
xl = np.linspace(10, 200, 500)
#print(xl)
for Cms_em, m_em in samples[np.random.randint(len(samples), size=100)]:
    pl.plot(xl, np.sqrt(sigQnr_c_v(xl)**2+(Cms_em+m_em*xl)**2), color="k", alpha=0.1)
pl.plot(xl, np.sqrt(sigQnr_c_v(xl)**2+(Cms+slope*xl)**2), color="r", lw=2, alpha=0.8)
#pl.errorbar(x, y, yerr=yerr, fmt=".k")
pl.xlim(10, 200)
pl.ylim(0.03,0.12)
pl.errorbar(E_recoil[2::], sig_NR[2::], yerr=sig_NR_err[2::], fmt=".k", capsize=0)

In [ ]:
sampsize=100
evec = np.zeros((np.shape(xl)[0],sampsize))
#print(samples[np.random.randint(len(samples), size=sampsize)])
i=0
for Cms_em, m_em in samples[np.random.randint(len(samples), size=sampsize)]:
    v = np.sqrt(sigQnr_c_v(xl)**2+(Cms_em+m_em*xl)**2)
    #print(np.std(v))
    #print(np.mean(v))
    cent = np.sqrt(sigQnr_c_v(xl)**2+(Cms+slope*xl)**2)
    evec[:,i] = v
    #print(i)
    #print(np.mean(v)+np.std(v))
    #print(np.mean(v)-np.std(v))
    i=i+1
   
upvec = np.mean(evec,axis=1)+np.std(evec,axis=1)
dnvec = np.mean(evec,axis=1)-np.std(evec,axis=1)

In [ ]:
pl.plot(xl, np.sqrt(sigQnr_c_v(xl)**2+(Cms+slope*xl)**2), color="r", lw=2, alpha=0.8)
pl.plot(xl, upvec, color="r",linestyle="--", lw=2, alpha=0.8)
pl.plot(xl, dnvec, color="r",linestyle="--", lw=2, alpha=0.8)
#pl.errorbar(x, y, yerr=yerr, fmt=".k")
pl.xlim(10, 200)
pl.ylim(0.03,0.12)
#args=(E_recoil[2::], sig_NR[2::], sig_NR_err[2::],sigQnr_c_v))
pl.errorbar(E_recoil[2::], sig_NR[2::], yerr=sig_NR_err[2::], fmt=".k", capsize=0)

In [ ]:
#save the results for the MS fit
path='{}/{}/'.format('mcmc','edwdata')

filename = 'data/mcmc_fits.h5'

#remove vars
f = h5py.File(filename,'a')
exCms = path+'Cms' in f
exm = path+'m' in f
exsamp = path+'samples' in f
exsampsize = path+'sampsize' in f
exEr = path+'Er' in f
exCsig_u = path+'Csig_u' in f
exCsig_l = path+'Csig_l' in f
exSigss = path+'Sigss' in f

if exCms:
  del f[path+'Cms']
if exm:
  del f[path+'m']
if exsamp:
  del f[path+'samples']
if exsampsize:
  del f[path+'sampsize']
if exEr:
  del f[path+'Er']
if exCsig_u:
  del f[path+'Csig_u']
if exCsig_l:
  del f[path+'Csig_l']
if exSigss:
  del f[path+'Sigss']

dset = f.create_dataset(path+'Cms',np.shape(Cms),dtype=np.dtype('float64').type)
dset[...] = Cms
dset = f.create_dataset(path+'m',np.shape(slope),dtype=np.dtype('float64').type)
dset[...] = slope
dset = f.create_dataset(path+'samples',np.shape(samples),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = samples
dset = f.create_dataset(path+'sampsize',np.shape(sampsize),dtype=np.dtype('float64').type)
dset[...] = sampsize
dset = f.create_dataset(path+'Er',np.shape(xl),dtype=np.dtype('float64').type, \
compression="gzip")
dset[...] = xl
dset = f.create_dataset(path+'Csig_u',np.shape(upvec),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = upvec
dset = f.create_dataset(path+'Csig_l',np.shape(dnvec),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = dnvec
dset = f.create_dataset(path+'Sigss',np.shape(xl),dtype=np.dtype('float64').type, \
compression="gzip",compression_opts=9)
dset[...] = sigQnr_c_v(xl)

f.close()